In [38]:
import numpy as np
import pandas as pd
import keras
import torch



In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
%run mrs.ipynb

# of rows of Song Data: 50683
# of unique songs: 50683
# of rows of User Listening Data: 806745
# of unique users: 25343
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50683 entries, 0 to 50682
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          50683 non-null  object 
 1   name              50683 non-null  object 
 2   artist            50683 non-null  object 
 3   spotify_id        50683 non-null  object 
 4   tags              49556 non-null  object 
 5   danceability      50683 non-null  float64
 6   energy            50683 non-null  float64
 7   loudness          50683 non-null  float64
 8   mode              50683 non-null  int64  
 9   speechiness       50683 non-null  float64
 10  acousticness      50683 non-null  float64
 11  instrumentalness  50683 non-null  float64
 12  liveness          50683 non-null  float64
 13  valence           50683 non-null  float64
 14  tempo        

C:\Users\cam-m\AppData\Local\Temp\ipykernel_9888\323123518.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\cam-m\AppData\Local\Temp\ipykernel_9888\323123518.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



69
[8 4 6 3 9 5 7 0 1 2]


In [40]:
#Note that the bert model and the mrs model need to run before this

x_test = np.load("misc/x_test.npy")
y_test = np.load("misc/y_test.npy")
model = keras.models.load_model("misc/mae_optimized_model.keras")
lyrics_embeddings_csv = pd.read_csv('misc/lyrics_embeddings.csv')
lyrics_embeddings_3d_csv = pd.read_csv('misc/lyrics_embeddings_3d.csv')
distance_frame = pd.read_csv('misc/distance_frame.csv')

lyrics_embeddings, lyrics_embeddings_3d = get_embeddings(lyrics_embeddings_csv, lyrics_embeddings_3d_csv)

C:\Users\cam-m\AppData\Local\Temp\ipykernel_9888\323123518.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\cam-m\AppData\Local\Temp\ipykernel_9888\323123518.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [41]:
POTENTIAL_N = 50

predicted = predict_sample(x_test[0], model)
recommendation = pipeline_helper(distance_frame, predicted, POTENTIAL_N, lyrics_embeddings)  
print(recommendation)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
[8 7 6 5 9 1 4 0 2 3]
                   name  danceability  energy  loudness  speechiness  \
2       Come as You Are         0.508   0.826    -5.783       0.0400   
7          Karma Police         0.360   0.505    -9.129       0.0260   
12       Feel Good Inc.         0.809   0.720    -7.776       0.1360   
18         Chasing Cars         0.558   0.568    -5.731       0.0267   
20  When You Were Young         0.466   0.981    -3.944       0.1120   
24            Starlight         0.555   0.880    -3.888       0.0312   
27         No Surprises         0.255   0.393   -10.654       0.0278   
34       Clint Eastwood         0.659   0.712    -7.602       0.1730   
40             Paranoid         0.413   0.772    -8.626       0.0778   
47     Heart-Shaped Box         0.609   0.842    -5.861       0.0342   

    acousticness  instrumentalness  liveness  valence  
2       0.000175          0.000459    0.0878    0.543  
7       0.062600          0.000092

### Spotify API Evaluation
Get recommendations from Spotify by passing in minimum and maximum feature values, and compare the songs to our recommended playlist.

In [ ]:
from spotify import SpotifyAPI
from dotenv import load_dotenv
# Get the input playlist
# Traverse along each song in playlist and preserve min max of feature values and 
# Store in list that gets passed to parameters for Spotify APIcommendations

input_playlist = og_songs
features = dict()
feature_names = ["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness", "valence"]

for feature in feature_names:
    features["min_" + feature] = input_playlist[feature].min()
    features["max_" + feature] = input_playlist[feature].max()

features["seed_genres"] = input_playlist['tags'].iloc[0].split(',')[:3]
features["seed_genres"] = ','.join(features["seed_genres"])
features

In [ ]:
env_path = os.path.join('misc', '.env')
load_dotenv(dotenv_path=env_path)
SPOTIFY_CREDS = [os.getenv('SPOTIFY_CLIENT_ID'), 
                os.getenv('SPOTIFY_CLIENT_SECRET')]

if not all(SPOTIFY_CREDS):
        print("Please set the SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET environment variables")
        exit(1)

spotify = SpotifyAPI(*SPOTIFY_CREDS)
SPOTIFY_CREDS

In [ ]:
recs_from_spotify = spotify.get_recommendations(features=features, limit=10)

In [ ]:
# SPOTIFY's PREDICTED RECOMMENDATIONS
recs_from_spotify = recs_from_spotify.loc[:, ['name', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                                'liveness', 'valence']]
recs_from_spotify

In [ ]:
def calculate_euclidean_distance(v1, v2):
    return np.linalg.norm(v1 - v2)

def calculate_score(pd1, pd2):
    if pd1.shape[1] != pd2.shape[1]:
        raise ValueError("Dataframes must have the same number of features.")
    
    for _, x in pd1.iterrows():
        fx = np.array(x[1:].values)
        dist = list()
        for _, y in pd2.iterrows():
            fy = np.array(y[1:].values)
            dist.append(calculate_euclidean_distance(fx, fy))
        print(np.mean(dist))